# Numerical Simulation of Ballistic Performance of Honeycomb Sandwich Panels

### Objective:
Setup a parametric model in PyMAPDL for a honeycomb sandwich panel. Use it for running a Ballistic impact simulation

The learning objectives of this demo are:
* Setup and solve a parametric model using PyMAPDL
* Interactive plotting of CAD, mesh, and results in Pythonic interface.

### Model Details

#### Model parameters:
* Cell size
* Cell wall thickness
* Node length
* Facesheet thickness

## Step 1 - define all parameters

In [25]:
## Main parameters
cell_size = 0.006
cell_wall_thickness = 0.002
node_length = 0.004
facesheet_thickness = 0.005

In [26]:
# Fixed constants
structure_length = 0.1
structure_breadth = 0.1

In [27]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# Additional calculation 
L = cell_size/ np.sqrt(3)
offset_x = cell_size + cell_wall_thickness
offset_y = 1.5*L + cell_wall_thickness*(np.sin(np.pi/3))
Nx = int(structure_length / offset_x) + 1
Ny = int(structure_breadth / offset_y) + 1
print("L = ", L)
print("offset_x = ", offset_x)
print("offset_y = ", offset_y)
print("Nx = ", Nx)
print("Ny = ", Ny)

L =  0.0034641016151377548
offset_x =  0.008
offset_y =  0.0069282032302755096
Nx =  13
Ny =  15


# Step 2 - launch MAPDL and create geometry

In [28]:
from ansys.mapdl.core import launch_mapdl

# start mapdl
mapdl = launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [29]:
## reset mapdl
mapdl.clear()

## enter the preprocessor
mapdl.prep7()

mapdl.units('MKS')      # MKS unit system

## Create geometry
mapdl.block(0, structure_length, 0, structure_breadth, 0, node_length)

plate = mapdl.cm("PLATE", "VOLU")

mapdl.clocal( 11, 0, structure_length/2, structure_breadth/2, 0)                                         ## Making local coordinate system
mapdl.wpcsys('', 11,)                                                                                    ## Shift working plane

initial_distance = node_length + 0.02                                              ## 20 mm from center of front facesheet of sandwich panel

# 0.44 Remington Magnum Bullet
# mapdl.cone(0.0053, 0.005805, initial_distance , initial_distance + 0.00825)
mapdl.cylind(0.005805,0, initial_distance + 0.00825 , initial_distance + 0.04089 )
# mapdl.vadd(1,2)
bullet = mapdl.cm("BULLET","VOLU")

mapdl.csdele(11)                                                                   ## Delete local coordinate system
# mapdl.input('bullet9.anf')

mapdl.vplot('all')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 3 - define material properties

In [30]:
# Define material properties
mapdl.mp("EX", 1, 2e5)  # Youngs modulus
mapdl.mp("PRXY", 1, 0.3367)  # Poissons ratio

mapdl.tb("EVISC",1)

## Define element attributes
mapdl.et(1, "SOLID185") # 3D 8-Node Layered Solid

1

In [31]:
# Saves the geometry in IGES format
mapdl.cdwrite("ALL",fname="C:/Users/HPRVa/Ballistic_shield_design_optimization/src/ geom",ext='igs')

*** MAPDL GLOBAL STATUS *** 

 TITLE =                                                                       
        16 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =             32
        22 LINES DEFINED         MAX LINE NUMBER =                 44
        10 AREAS DEFINED         MAX AREA NUMBER =                 20
         2 VOLUMES DEFINED       MAX VOLUME NUMBER =                2
         1 ELEM TYPES DEFINED    MAX ELEM TYPE NUMBER =             1
         0 ELEMENTS DEFINED      MAX ELEMENT NUMBER =               0
         0 NODES DEFINED         MAX NODE NUMBER =                  0

 WRITE ANSYS DATABASE AS AN ANSYS INPUT FILE: C:/Users/HPRVa/Ballistic_shield_design_optimization/src/ geom.igs                                                                                                                                                                                                   

 WRITE IGES FILE= C:/Users/HPRVa/Ballistic_shield_design_optimization/src/

 ATTRIBUTES WIL

In [32]:
mapdl.exit()

## Launch Ansys Mechanical 

In [33]:
import json
import os
import grpc

from ansys.mechanical.core import launch_mechanical

mechanical = launch_mechanical(batch=True, cleanup_on_exit=False)
print(mechanical)

Ansys Mechanical [Ansys Mechanical Enterprise]
Product Version:231
Software build date:Wednesday, November 23, 2022 6:49:22 PM


In [34]:
# geometry_path = "geom.iges"

In [35]:
# # project_directory = mechanical.project_directory
# project_directory = "C:/Users/HPRVa/Ballistic_shield_design_optimization/src"
# print(f"project directory = {project_directory}")

# # Upload the file to the project directory.
# # mechanical.upload(file_name=geometry_path, file_location_destination=project_directory)

# # Build the path relative to project directory.
# base_name = os.path.basename(geometry_path)
# combined_path = os.path.join(project_directory, base_name)
# part_file_path = combined_path.replace("\\", "\\\\")
# mechanical.run_python_script(f"part_file_path='{part_file_path}'")

# # Verify the path
# result = mechanical.run_python_script("part_file_path")
# print(f"part_file_path on server: {result}")

In [36]:
output = mechanical.run_python_script(
    r"""
import json

#region Context Menu Action
geometry_import_group_11 = Model.GeometryImportGroup
geometry_import_13 = geometry_import_group_11.AddGeometryImport()

geometry_import_13_format = Ansys.Mechanical.DataModel.Enums.GeometryImportPreference.\
    Format.Automatic
geometry_import_13_preferences = Ansys.ACT.Mechanical.Utilities.GeometryImportPreferences()
geometry_import_13_preferences.ProcessNamedSelections = True
geometry_import_13_preferences.ProcessCoordinateSystems = True
# geometry_import_13 = DataModel.GetObjectById(13)
geometry_import_13.Import(r"C:\Users\HPRVa\Ballistic_shield_design_optimization\src\ geom.iges") # Primary Source!
#endregion
#region Context Menu Action
ExtAPI.Application.ActiveUnitSystem = MechanicalUnitSystem.StandardMKS
#endregion


#region Details View Action
body_25 = DataModel.GetObjectById(25)
body_25.StiffnessBehavior = StiffnessBehavior.Rigid
#endregion

#region Toolbar Action
model_11 = Model
analysis_29 = model_11.AddExplicitDynamicsAnalysis()
#endregion

#region Context Menu Action
initial_conditions_34 = DataModel.GetObjectById(34)
initial_condition_37 = initial_conditions_34.InsertVelocity()

selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [55]
initial_condition_37.Location = selection
#endregion

#region Details View Action
initial_condition_37.DefineBy = LoadDefineBy.Components
#endregion

#region Details View Action
initial_condition_37.ZComponent = Quantity(-400, "m/s")
#endregion

solution_30 = analysis_29.Solution

#region Context Menu Action
total_deformation_41 = solution_30.AddTotalDeformation()

selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [32]
total_deformation_41.Location = selection
#endregion

#region Context Menu Action
velocity_probe_43 = solution_30.AddVelocityProbe()
#endregion

#region Details View Action
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [55]
velocity_probe_43.GeometryLocation = selection
#endregion

#region Details View Action
velocity_probe_43.ResultSelection = ProbeDisplayFilter.ZAxis
#endregion

#region Context Menu Action
energy_probe_45 = solution_30.AddEnergyProbe()
#endregion

#region Details View Action
selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
selection.Ids = [55]
energy_probe_45.GeometryLocation = selection
#endregion

#region Details View Action
mesh_15 = Model.Mesh
mesh_15.PhysicsPreference = MeshPhysicsPreferenceType.Explicit
#endregion

#region Details View Action
mesh_15.ElementSize = Quantity(0.001, "m")
#endregion

#region Context Menu Action
mesh_15.GenerateMesh()
#endregion

# #region UI Action
# with Transaction(True):
#     body_22 = DataModel.GetObjectById(22)
#     body_22.Material = "cc89e716-9563-431b-98d0-b698ebff6d55"
# #endregion

# #region UI Action
# with Transaction(True):
#     body_25 = DataModel.GetObjectById(25)
#     body_25.Material = "ce7f3c8f-cb53-4f3e-9517-ac8ade5cd762"
# #endregion

# analysis_29.ErosionOnMaterialFailureType=True

# analysis_29.SetStepEndTime(1,Quantity(1e-04, "s"))
# analysis_29.StepsEndTime

# Solve and get the results.

# Solve Explicit Dynamics Analysis
analysis_29.Solution.Solve(True)

 """
)
print(output)

In [24]:
mechanical.exit()